In [1]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

from collections import defaultdict

from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import HdpModel
from gensim.models import CoherenceModel

In [3]:
lemma_grouped = pd.read_pickle("../data/df_lemma_grouped_from2018.pkl")

In [4]:
lemma_grouped = lemma_grouped[lemma_grouped.apply(len) >= 10]

In [5]:
lemma_grouped.info()

<class 'pandas.core.series.Series'>
Index: 2538 entries, 396_1. Mitteilungen to 776_9. Standesinitiative zum sofortigen Stopp der Sanktionen gegen Russland und zur sofortigen Wiederinkraftsetzung der immerwährenden Neutralität der Schweiz, gemäss dem Wienerkongress von 1815
Series name: lemma
Non-Null Count  Dtype 
--------------  ----- 
2538 non-null   object
dtypes: object(1)
memory usage: 39.7+ KB


In [6]:
docs = lemma_grouped

In [7]:
frequency = defaultdict(int)
for text in list(docs):
    for token in set(text):
        frequency[token] += 1
frequency

defaultdict(int,
            {'absolvierten': 5,
             'Limmattal': 61,
             'Abgaben': 207,
             'Claudia': 108,
             'Regierungsrat': 1489,
             'René': 132,
             'SP': 1330,
             'Rathüsler': 1,
             'Glattfelden': 118,
             'Dalcher': 128,
             'Kredits': 32,
             'Jörg': 161,
             'Steiner': 204,
             'BDP': 121,
             'nutzen': 406,
             'Ratsprotokoll': 125,
             'Astrid': 77,
             'Daniel': 413,
             'Laufzeit': 20,
             'Sozialhilfemissbrauch': 8,
             'Vorzeitige': 2,
             'Minuten': 323,
             'Tobias': 199,
             'Bettina': 109,
             'Umsetzung': 707,
             'Marcel': 157,
             'Applaus': 181,
             'Bedingungen': 272,
             '155.': 2,
             'Kindergarteneintritt': 7,
             'Heiterkeit': 316,
             'Pflugshaupt': 78,
             'Spitälern'

In [8]:
sorted_frequency = sorted(frequency.items(), key=lambda x:x[1], reverse=True)
for f in sorted_frequency:
    if f[1]/len(docs) < 0.05:
        break
    print(f"{f[0]}: {f[1]/len(docs)}")

Zürich: 0.7572892040977147
Geschäft: 0.7470449172576832
erledigt: 0.7261623325453113
KR-Nr: 0.7206461780929866
Antrag: 0.6765169424743893
Kanton: 0.5921985815602837
Regierungsrat: 0.586682427107959
Kantonsrat: 0.5819542947202522
SVP: 0.5732860520094563
Fall: 0.5669818754925138
FDP: 0.5472813238770685
SP: 0.5240346729708432
Stimmen: 0.516548463356974
Kommission: 0.48778565799842394
Vorlage: 0.48739164696611503
einfach: 0.4511426319936958
gestellt: 0.44129235618597323
Abstimmung: 0.438140267927502
Kantons: 0.43656422379826637
bitte: 0.4318360914105595
klar: 0.4306540583136328
beantragt: 0.42750197005516155
braucht: 0.4247438928289992
gehört: 0.4168636721828211
sagen: 0.4152876280535855
Regierungsrates: 0.41174152876280534
Zürcher: 0.4066193853427896
unterstützen: 0.4030732860520095
Paragraf: 0.3963750985027581
genau: 0.39440504334121357
unterstützt: 0.39440504334121357
letzten: 0.39243498817966904
genehmigt: 0.39204097714736014
wichtig: 0.3908589440504334
ebenfalls: 0.3892828999211978
nä

In [9]:
# Compute bigrams.

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [10]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.10)

In [11]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [12]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 20698
Number of documents: 2538


In [13]:
hdp = HdpModel(corpus, dictionary, alpha=0.01, eta=0.01) # 0.5382 | 98
#hdp = HdpModel(corpus, dictionary, alpha=0.01, eta=0.0001) # 0.5533 | 102
#hdp = HdpModel(corpus, dictionary, alpha=0.75, eta=0.0001) # 0.5316 | 114
#hdp = HdpModel(corpus, dictionary)  # 0.5395 | 113. 

In [14]:
topic_info = hdp.print_topics(num_topics=20, num_words=10)

In [15]:
len(hdp.get_topics())

150

In [16]:
hdp.print_topics(-1)

[]

In [17]:
for i in range(len(hdp.get_topics())):
    print(i)
    print(hdp.print_topic(i))

0
0.001*"KEF-Erklärung" + 0.001*"USZ" + 0.001*"CO" + 0.001*"EKZ" + 0.001*"Frauen" + 0.001*"Lehrpersonen" + 0.001*"ZKB" + 0.001*"Leistungsgruppe" + 0.001*"−" + 0.001*"Universität"
1
0.002*"Universität" + 0.002*"Menschenhandel" + 0.002*"materielle" + 0.002*"Richtplan" + 0.002*"−" + 0.002*"Einzelinitiative" + 0.001*"KR-Nr_Entgegennahme" + 0.001*"Postulat_entgegenzunehmen" + 0.001*"materielle_Behandlung" + 0.001*"E-Voting"
2
0.003*"Wahlzettel" + 0.002*"Wasser" + 0.002*"Stimmzettel" + 0.002*"Stimmenzahl" + 0.002*"Stimmenzähler" + 0.002*"Gewässer" + 0.002*"Anlagen" + 0.002*"−" + 0.002*"Interfraktionelle" + 0.002*"Interfraktionellen"
3
0.004*"Finanzkommission" + 0.003*"KEF" + 0.003*"Milliarden" + 0.002*"Ausgleich" + 0.002*"Milliarden_Franken" + 0.002*"FIKO" + 0.002*"Lü16" + 0.001*"Budgets" + 0.001*"Leistungsgruppe" + 0.001*"Steuerfuss"
4
0.003*"Wasserversorgung" + 0.002*"−" + 0.002*"Wassergesetz" + 0.002*"Hunde" + 0.002*"Hund" + 0.002*"Kirche" + 0.002*"Rathaus" + 0.002*"Religionsgemeinschafte

In [18]:
topic_info

[(0,
  '0.001*KEF-Erklärung + 0.001*USZ + 0.001*CO + 0.001*EKZ + 0.001*Frauen + 0.001*Lehrpersonen + 0.001*ZKB + 0.001*Leistungsgruppe + 0.001*− + 0.001*Universität'),
 (1,
  '0.002*Universität + 0.002*Menschenhandel + 0.002*materielle + 0.002*Richtplan + 0.002*− + 0.002*Einzelinitiative + 0.001*KR-Nr_Entgegennahme + 0.001*Postulat_entgegenzunehmen + 0.001*materielle_Behandlung + 0.001*E-Voting'),
 (2,
  '0.003*Wahlzettel + 0.002*Wasser + 0.002*Stimmzettel + 0.002*Stimmenzahl + 0.002*Stimmenzähler + 0.002*Gewässer + 0.002*Anlagen + 0.002*− + 0.002*Interfraktionelle + 0.002*Interfraktionellen'),
 (3,
  '0.004*Finanzkommission + 0.003*KEF + 0.003*Milliarden + 0.002*Ausgleich + 0.002*Milliarden_Franken + 0.002*FIKO + 0.002*Lü16 + 0.001*Budgets + 0.001*Leistungsgruppe + 0.001*Steuerfuss'),
 (4,
  '0.003*Wasserversorgung + 0.002*− + 0.002*Wassergesetz + 0.002*Hunde + 0.002*Hund + 0.002*Kirche + 0.002*Rathaus + 0.002*Religionsgemeinschaften + 0.001*Hundegesetz + 0.001*Paragrafen'),
 (5,
  '0

In [19]:
coherence_model_hdp = CoherenceModel(model=hdp, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_hdp = coherence_model_hdp.get_coherence()
print('Coherence Score: ', coherence_hdp)

Coherence Score:  0.5382091793374472
